In [ ]:
# first we need to install some needed libraries

# Since machine can not read from pdf file directly we need to convert it into machine readable format like text,html,json (here we convert it to text)
!pip install -q pypdf
!pip install -q python-dotenv
# transformer will be used to access the models for embedding and using LLM
!pip install -q transformers

In [ ]:
# installing llama architecture.
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423614 sha256=8720d79e1100566522656ff85844f9ebf766cb36b4183f7ce88b37ac7b34f459
  Stored in directory: /tmp/pip-ephem-wheel-cache-_c5oxlv7/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [ ]:
# Installing llama-index that Connect the data we get from the UGRULE-book pdf to the LLM.
!pip install -q llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.6/784.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
# now we will import three libraries from llama index
# 1. Vectorspace : the documents we will provide will be converted into texts then into chunks and then into vectors as embedding these vectors need to be stored which is done by this library
# 2. SimpleDirectoryReader this library will be used to read the UG-rulebook pdf file
# 3. ServiceContext : this provides context for interacting with the LlamaIndex library.It includes information and configuration about the LLM and the embedding model we will use
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [ ]:
# To read our ug-rulebook pdf file
documents = SimpleDirectoryReader("/content").load_data()

In [ ]:
# Pytorch will help us to work using GPU.
import torch
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # URL to GGLM model to downlioad it and use it
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',

    # to controls the randomness of the generated text . Higher temperatures mean more creative and less predictable text we put it 0.1 to make it more formal and trustable
    temperature=0.1,

    # maximim number of token that can be generated by LLM
    max_new_tokens=256,

    # number of tokens that will be used to generate the next token.
    context_window=3900,

    # kwargs to pass to __call__()
    generate_kwargs={},

    # kwargs to pass to __init__()
    # This tells LlamaCPP to use all available GPUs.
    model_kwargs={"n_gpu_layers": -1},

    # to convert messages and completions into the format that is expected by LlamaCPP
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    # making verbose true mean that the model will generate more workd than neccessary.
    verbose=True,
);


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# library we will need to do embeddings of the document
!pip -q install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00


In [ ]:
# Embedding part
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding
 # model we will use to embed
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

In [ ]:
# defining how the chunks will be made like chunk size , chunk_overlap and model used etc
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# The from_documents() method takes a list of documents and a ServiceContext object as input and returns a VectorStoreIndex object. The VectorStoreIndex object will contain the vector representations of the documents in the input list.
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
# QueryEngine provides a more efficient and user-friendly way to query a VectorStoreIndex object.
query_engine = index.as_query_engine()

In [ ]:
# to distplay the output in paragraph rather than a long scrollable line
from IPython.display import HTML

In [ ]:
# querying and answering
i=0
while True:
  query=input()
  response = query_engine.query(query)
  i+=1
  display(HTML(f"<h3>query number {str(i)}: {str(query)} </h3>" + "<br>"+ str(response)))

what is IDDDP prgramme?


Llama.generate: prefix-match hit


How are NPTEL/SWAYAM course credits equated with IIT Bombay courses? 


Llama.generate: prefix-match hit


What are the requirements for earning a degree?


...

Llama.generate: prefix-match hit


आईडीडीपी क्या है?


Llama.generate: prefix-match hit


आईडीडीडीपी क्या है?


Llama.generate: prefix-match hit
